In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
import seaborn as sns
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)


PATH = './data/'

In [9]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_quality  = pd.read_csv(PATH+'train_quality_data.csv')
train_problem  = pd.read_csv(PATH+'train_problem_data.csv')

test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_quality  = pd.read_csv(PATH+'test_quality_data.csv')

In [18]:
# train_quality 데이터에 대해서 strin2num 함수 적용 int로 변환
train_quality['quality_10'] = train_quality['quality_10'].apply(string2num)

In [99]:
# quality 10 에 대해서 행값 평균이 200이 넘는 경우 추출
train_quality.groupby('user_id').quality_10.mean()[train_quality.groupby('user_id').quality_10.mean() >= 200]

user_id
10019    8299.315789
10032     669.235294
10057     259.500000
10062     452.000000
10077    5543.666667
            ...     
24964    7972.611111
24979    1001.750000
24981    2494.750000
24984    2048.500000
24986     332.125000
Name: quality_10, Length: 950, dtype: float64

In [108]:
# 저장
filter_train=train_quality.groupby('user_id').quality_10.mean()>=200

In [109]:
# 인덱스 값
filter_train.index[filter_train==True]

Int64Index([10019, 10032, 10057, 10062, 10077, 10081, 10089, 10090, 10150,
            10166,
            ...
            24892, 24900, 24920, 24922, 24934, 24964, 24979, 24981, 24984,
            24986],
           dtype='int64', name='user_id', length=950)

In [110]:
# 인덱스 리스트 저장
q10train_list=[x for x in filter_train.index[filter_train==True]]

In [111]:
# 넘파이로 200이 넘는 경우 1 과 0 으로 표시
q10train_check = np.zeros((15000,1)) # 15000 아이디에 대해서 0 열 만들기

for idx in range(15000):
    if np.isin(idx+10000, [q10train_list]): # 10000부터 아이디가 시작하므로
        q10train_check[idx] = 1 #인덱스 리스트에 있으면 1 지정

In [113]:
q10train_check.sum()

950.0

In [72]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

In [73]:
##시간별 데이터 만드는 코딩
def time_encoding(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    day   = int(x[6:8])
    hour  = int(x[8:10])
    y = day*100 + hour
    for i in range(30):
        if y == (i+1)*100:
            return 0
        if y == (i+1)*100 + 1:
            return 1
        if y == (i+1)*100 + 2:
            return 2
        if y == (i+1)*100 + 3:
            return 3
        if y == (i+1)*100 + 4:
            return 4
        if y == (i+1)*100 + 5:
            return 5
        if y == (i+1)*100 + 6:
            return 6
        if y == (i+1)*100 + 7:
            return 7
        if y == (i+1)*100 + 8:
            return 8
        if y == (i+1)*100 + 9:
            return 9
        if y == (i+1)*100 + 10:
            return 10
        if y == (i+1)*100 + 11:
            return 11
        if y == (i+1)*100 + 12:
            return 12
        if y == (i+1)*100 + 13:
            return 13
        if y == (i+1)*100 + 14:
            return 14
        if y == (i+1)*100 + 15:
            return 15
        if y == (i+1)*100 + 16:
            return 16
        if y == (i+1)*100 + 17:
            return 17
        if y == (i+1)*100 + 18:
            return 18
        if y == (i+1)*100 + 19:
            return 19
        if y == (i+1)*100 + 20:
            return 20
        if y == (i+1)*100 + 21:
            return 21
        if y == (i+1)*100 + 22:
            return 22
        if y == (i+1)*100 + 23:
            return 23
        if day == 31: # 10월 31일 23:59의 경우
            return 0

In [74]:
#적용하기
train_err['timeslot'] = train_err['time'].apply(time_encoding)

In [75]:
# 시간대별로 타임라벨링 하기 넘파이 활용
timeslot_label = train_err[['user_id','timeslot']].values
timeslot_check = np.zeros((train_user_number,len(train_err['timeslot'].unique())))

for person_idx, ch in tqdm(timeslot_label):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    timeslot_check[person_idx - train_user_id_min,ch - 1] += 1
timeslot_check.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:45<00:00, 364975.59it/s]


(15000, 24)

In [76]:
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:46<00:00, 359338.15it/s]


(15000, 42)

In [79]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


# 숫자 값으로 변환하기 위해 LabelEncoder로 먼저 변환한다.
### 모델명에 대해서 라벨 인코딩 진행
encoder = LabelEncoder()
encoder.fit(train_err['model_nm'].astype('|S80')) ## object 변수이므로 str 변수로 변환
labels_model = encoder.transform(train_err['model_nm'].astype('|S80'))
train_err = pd.concat([train_err,pd.Series(labels_model)],axis=1)
train_err.rename(columns = {0 : 'labels_model'}, inplace = True)

In [120]:
model_label = train_err[['user_id','labels_model']].values
model_nm = np.zeros((train_user_number,9))

for person_idx, ch in tqdm(model_label):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    model_nm[person_idx - train_user_id_min,ch - 1] += 1
model_nm_ = (model_nm > 0).astype(int) #0보다 큰 데이터는 그냥 전부 1로 변경
model_nm_.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:49<00:00, 332810.77it/s]


(15000, 9)

In [121]:
# 정리
# q10train_check - 퀄리티10 데이터가 평균이 200이 넘는 경우, 데이터프레임
# timeslot_check - 시간대별 데이터
# error - 에러타입별 데이터
# model_nm_ - 모델별 데이터

In [82]:
# train_quality 데이터에 대해서 strin2num 함수 적용 int로 변환
test_quality['quality_10'] = test_quality['quality_10'].apply(string2num)

# quality 10 에 대해서 행값 평균이 200이 넘는 경우 추출
test_quality.groupby('user_id').quality_10.mean()[test_quality.groupby('user_id').quality_10.mean() >= 200]

user_id
30001     854.333333
30005    1580.937500
30008     458.571429
30009    3086.333333
30061     718.500000
            ...     
44949     978.100000
44956    4532.000000
44971     425.513514
44978    2221.000000
44985     355.000000
Name: quality_10, Length: 934, dtype: float64

In [124]:
# 저장
filter_test=test_quality.groupby('user_id').quality_10.mean()>=200
# 인덱스 값
filter_test.index[filter_test==True]
# 인덱스 리스트 저장
q10test_list=[x for x in filter_test.index[filter_test==True]]

In [125]:
# 넘파이로 200이 넘는 경우 1 과 0 으로 표시
q10test_check = np.zeros((14999,1)) # 15000 아이디에 대해서 0 열 만들기

for idx in range(14999):
    if np.isin(idx+30000, [q10test_list]): # 10000부터 아이디가 시작하므로
        q10test_check[idx] = 1 #인덱스 리스트에 있으면 1 지정

In [127]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [128]:
# 시간대 나누는 함수 적용하기
test_err['timeslot'] = test_err['time'].apply(time_encoding)

In [129]:
# 시간대별로 타임라벨링 하기 넘파이 활용
timeslot_label_test = test_err[['user_id','timeslot']].values
timeslot_check_test = np.zeros((test_user_number,len(test_err['timeslot'].unique())))

for person_idx, ch in tqdm(timeslot_label_test):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    timeslot_check_test[person_idx - test_user_id_min,ch - 1] += 1
timeslot_check_test.shape

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:48<00:00, 344010.66it/s]


(14999, 24)

In [134]:
id_error_test = test_err[['user_id','errtype']].values
error_test = np.zeros((test_user_number,42))

for person_idx, err in tqdm(id_error_test):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error_test[person_idx - test_user_id_min,err - 1] += 1
error_test.shape

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:49<00:00, 334740.89it/s]


(14999, 42)

In [135]:
# 숫자 값으로 변환하기 위해 LabelEncoder로 먼저 변환한다.
### 모델명에 대해서 라벨 인코딩 진행
encoder = LabelEncoder()
encoder.fit(test_err['model_nm'].astype('|S80')) ## object 변수이므로 str 변수로 변환
labels_model_test = encoder.transform(test_err['model_nm'].astype('|S80'))
test_err = pd.concat([test_err,pd.Series(labels_model_test)],axis=1)
test_err.rename(columns = {0 : 'labels_model_test'}, inplace = True)

MemoryError: Unable to allocate 1.23 GiB for an array with shape (16532648,) and data type |S80

In [ ]:
model_label_test = test_err[['user_id','labels_model_test']].values
model_nm_test = np.zeros((test_user_number,9))

for person_idx, ch in tqdm(model_label_test):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    model_nm_test[person_idx - test_user_id_min,ch - 1] += 1
model_nm_test_ = (model_nm > 0).astype(int) #0보다 큰 데이터는 그냥 전부 1로 변경
model_nm_test_.shape